
For constant free stream flow, the [Blasius similarity solution](https://en.wikipedia.org/wiki/Blasius_boundary_layer#Blasius_equation_-_First-order_boundary_layer) gives the boundary layer behavior a s a function of the similarity variable $$\eta = \frac{y}{\delta(x)} = y \sqrt{\frac{U_\infty}{ \nu x}},$$
where $\delta(x)$ is the boundary layer thickness. Under Blasius' assumptions the flow is determined by the solution to the nonlinear boundary value problem
$$ f'''(\eta) + \frac{1}{2}f''(\eta) f(\eta), $$
$$ f(0) = f'(0) = 0, f'(\infty) = 1$$

Since this is a third-order equation, if a value for $f''(0)$ can be found which satisfies the infinite boundary condition, the self-similarity function $f(\eta)$ can be found numerically (e.g. with Runge-Kutta integration). This unknown "initial condition" can be established through root-finding.

Once $f(\eta)$ is known, the boundary layer profile can be found via
$$ u(x, y) = U_\infty f'(\eta), \hspace{1cm} v(x, y) = \frac{U_\infty}{2}\sqrt{\frac{\nu}{ U_\infty x}} [\eta f'(\eta) - f(\eta)]. $$
In other words, $u/U_\infty$ depends only on $\eta$, but $v/U_\infty$ depends on $\eta$ and $Re_x$

Obviously the equation cannot be solved numerically at infinity, but since it quickly approaches a linear solution, solving the problem on $\eta \in (0, 10)$ should be sufficient.

To solve, we need to define variables to reduce the third-order ODE to a first-order system of ODEs. This is done by defining
$$ f(\eta) = f_0 $$
$$ f'(\eta) = f_1 $$
$$ f''(\eta) = f_2 $$
Then the BVP becomes
$$f_0' = f_1 $$
$$ f_1' = f_2 $$
$$ f_2' = -\frac{1}{2}f_2 f_0 $$
with boundary conditions
$$f_0(0) = f_1(0) = 0$$
$$f_1(\infty) = 1$$

To solve with scipy, need to define a function to evaluate the ODE (for forward integration) and a function to evaluate the error in the boundary conditions:


In [5]:
import numpy as np
import numpy.random as rng
from scipy.integrate import odeint

import sys
sys.path.append('../solvers/')
sys.path.append('../src/')

from blasius import solve_blasius
from learning import KRidgeReg
from helper_functions import prettify_results

In [6]:
eta_inf=10
d_eta=0.01
nu = 1e-6       # Viscosity of water near room temperature  (m^2/s)
U_inf = 0.01    # m/s
xlim = [1e-3, 1e-1]

x, y, u, v = solve_blasius(xlim, nu, U_inf, eta_inf, d_eta)

yy, xx = np.meshgrid(y, x)
eta = yy*np.sqrt(U_inf/(xx*nu))  # Exact value of eta

Re = (U_inf/nu)*x
delta = 1.72 * np.sqrt(x*nu/U_inf)

# Optimization

In [7]:

dim_matrix = np.array([[1, 0], [1, 0], [1, -1], [2,  -1]]).T
names = ['x', 'y', 'U_{inf}', '\\nu']

# True vectors are in the nullspace
eta_vec = np.array([-0.5, 1, 0.5, -0.5])
Re_vec = np.array([1, 0, 1, -1])
print('$\eta$ and $Re$ are in null-space:')
print(dim_matrix @ eta_vec)
print(dim_matrix @ Re_vec)

# Inputs: [x, y, U, nu]
p = np.vstack([xx.flatten(),
               yy.flatten(),
               np.full(u.shape, U_inf).flatten(),
               np.full(u.shape, nu).flatten()]).T

# Outputs: [u/Uinf, v/U_inf]
q = np.vstack([u.flatten()/U_inf, v.flatten()/U_inf])

# Randomly subsample observable and parameters
nsamp = 500
idx = np.random.choice(np.arange(q.shape[1]), nsamp)
outputs = q[:, idx].T
inputs = p[idx, :]

$\eta$ and $Re$ are in null-space:
[0. 0.]
[0 0]


In [8]:
## Consider hyperparameter optimization over l1_reg and alpha
l1_reg = 1e-6 ## Solution is sensitive to L1
alpha = 1e-4
kernel = 'rbf'
gamma = 30
use_test_set = True
normalize = True
num_trials = 30
frac_tol = 0.1
max_denominator = 10
num_nondim = 2

# Ridge Regression
K = KRidgeReg(inputs, outputs, dim_matrix, num_nondim=num_nondim, #normalize=normalize,
 l1_reg=l1_reg, alpha=alpha, kernel=kernel, gamma=gamma)

x, x_list, loss_list = K.multi_run(ntrials=num_trials)

pi1 = x[:, 0]/min(x[:, 0])
pi2 = x[:, 1]/min(x[:, 1])

prettify_results(pi1, names, tol=frac_tol, max_degree=max_denominator)
prettify_results(pi2, names, tol=frac_tol, max_degree=max_denominator)


  3%|▎         | 1/30 [00:00<00:23,  1.22it/s]

0.03304542246637743
[[ 1.72511906 -4.89422863]
 [ 3.73678139  3.85797104]
 [ 5.46190046 -1.03625758]
 [-5.46190046  1.03625758]]


  7%|▋         | 2/30 [00:01<00:24,  1.13it/s]

0.03296600898351287
[[-2.16253561 -5.92840418]
 [-2.08783914  4.70055814]
 [-4.25037475 -1.22784604]
 [ 4.25037475  1.22784604]]


 10%|█         | 3/30 [00:05<00:42,  1.56s/it]

...

100%|██████████| 30/30 [00:46<00:00,  1.56s/it]

[-0.31584594 -0.68415406 -1.          1.        ]
x : -0.3333333333333333
y : -0.6666666666666666
U_{inf} : -1
\nu : 1


<IPython.core.display.Math object>

[ 1.         -0.78826948  0.21173052 -0.21173052]
x : 1
y : -0.75
U_{inf} : 0.25
\nu : -0.25


<IPython.core.display.Math object>

In [9]:

dim_matrix = np.array([[1, 0], [1, 0], [1, -1], [2,  -1]]).T
names = ['x', 'y', 'U_{inf}', '\\nu']

# True vectors are in the nullspace
eta_vec = np.array([-0.5, 1, 0.5, -0.5])
Re_vec = np.array([1, 0, 1, -1])
print('$\eta$ and $Re$ are in null-space:')
print(dim_matrix @ eta_vec)
print(dim_matrix @ Re_vec)

# Inputs: [x, y, U, nu]
p = np.vstack([xx.flatten(),
               yy.flatten(),
               np.full(u.shape, U_inf).flatten(),
               np.full(u.shape, nu).flatten()]).T

# Outputs: [u/Uinf, v/U_inf]
q = np.vstack([u.flatten()/U_inf, v.flatten()/U_inf])

# Randomly subsample observable and parameters
nsamp = 100
idx = np.random.choice(np.arange(q.shape[1]), nsamp)
outputs = q[: idx].T
inputs = p[idx, :]

$\eta$ and $Re$ are in null-space:
[0. 0.]
[0 0]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
## Consider hyperparameter optimization over l1_reg and alpha
l1_reg = 1e-6 ## Solution is sensitive to L1
alpha = 1e-4
kernel = 'rbf'
gamma = 1 
use_test_set = True
normalize = True
num_trials = 80
frac_tol = 0.1
max_denominator = 10
num_nondim = 1

# Ridge Regression
K = KRidgeReg(inputs, outputs, dim_matrix, num_nondim=num_nondim, #normalize=normalize,
 l1_reg=l1_reg, alpha=alpha, kernel=kernel, gamma=gamma)

x, x_list, loss_list = K.multi_run(ntrials=num_trials)

pi1 = x[:, 0]/x[1, 0]

prettify_results(pi1, names, tol=frac_tol, max_degree=max_denominator)


0.02710798227229194
0.027095360444421845
[-0.03855079  1.          0.96144921 -0.96144921]
x : 0
y : 1
U_{inf} : 1
\nu : -1


<IPython.core.display.Math object>

In [ ]:
x_list_arr = []
x_list_arr_idx = []
xdiv = 0
for idx, x in enumerate(x_list):
    if x is not None:
        if type(x) != np.ndarray:
            xdiv += 1
        else:
            x_list_arr.append(x)    
            x_list_arr_idx.append(idx)

x_list_arr = x_list_arr
x_list_arr_idx = np.array(x_list_arr_idx)

num_none = len(x_list) - len(x_list_arr)

x_list_norm = []
for x in x_list_arr:
    x_list_norm.append( x[:, 0]/min(x[:, 0]) )

loss_list_arr = np.array(loss_list)
loss_list_arr[x_list_arr_idx]


array([0.02710798, 0.02709536])

In [ ]:
x_list_norm

[array([-0.03855079,  1.        ,  0.96144921, -0.96144921]),
 array([ 0.0406699, -1.0406699, -1.       ,  1.       ])]

In [11]:
import sklearn
sklearn.metrics.pairwise.PAIRWISE_KERNEL_FUNCTIONS

{'additive_chi2': <function sklearn.metrics.pairwise.additive_chi2_kernel(X, Y=None)>,
 'chi2': <function sklearn.metrics.pairwise.chi2_kernel(X, Y=None, gamma=1.0)>,
 'linear': <function sklearn.metrics.pairwise.linear_kernel(X, Y=None, dense_output=True)>,
 'polynomial': <function sklearn.metrics.pairwise.polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1)>,
 'poly': <function sklearn.metrics.pairwise.polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1)>,
 'rbf': <function sklearn.metrics.pairwise.rbf_kernel(X, Y=None, gamma=None)>,
 'laplacian': <function sklearn.metrics.pairwise.laplacian_kernel(X, Y=None, gamma=None)>,
 'sigmoid': <function sklearn.metrics.pairwise.sigmoid_kernel(X, Y=None, gamma=None, coef0=1)>,
 'cosine': <function sklearn.metrics.pairwise.cosine_similarity(X, Y=None, dense_output=True)>}